In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/amazon-electronics"))
from sklearn.model_selection import train_test_split

# Any results you write to the current directory are saved as output.

In [ ]:
ele=pd.read_csv('../input/amazon-electronics/ratings_Electronics.csv',names=['reviewerID','Product_ID','rating','timestamp'])

In [ ]:
ele.shape

In [ ]:
#remove timestamp
ele=ele.drop(columns='timestamp')
ele.head()

In [ ]:
ele['reviewerID'].unique().shape

In [ ]:
ele.isnull().sum()

In [ ]:
above_fifty=ele.groupby(['reviewerID']).agg('count').sort_values(by='rating',ascending=False)

In [ ]:
above_fifty_id=above_fifty[above_fifty['rating']>50].index

In [ ]:
above_fifty_id

In [ ]:
Final_df=ele[ele['reviewerID'].isin(above_fifty_id.values)]

In [ ]:
Final_df.shape

In [ ]:
Final_df=(Final_df.reset_index())

In [ ]:
Final_df=Final_df.drop(columns='index')

In [ ]:
Final_df.head()

In [ ]:
Train,Test=train_test_split(Final_df,train_size=0.75,random_state=1)

In [ ]:
Train.shape

In [ ]:
R_df=Train.pivot(index = 'reviewerID', columns ='Product_ID', values = 'rating').fillna(0)

In [ ]:
R_df.head()

In [ ]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_df, k = 50)

In [ ]:
sigma = np.diag(sigma)

In [ ]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [ ]:
#lets find the recomondation for reviewerID=AT09WGFUM934H
reviewerID='A32HSNCNPRUMTR'
user_id = 2

In [ ]:
 # Get and sort the user's predictions
user_row_number = user_id - 1 # UserID starts at 1, not 0
sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False)

In [ ]:
##Top 5 Product recmondations is 
sorted_user_predictions.head(5).index.values

# Popularity based recomondation

In [ ]:
class popularity_recommender_py():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        self.popularity_recommendations = None
        
    #Create the popularity based recommender system model
    def create(self, train_data, user_id, item_id):
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id

        #Get a count of user_ids for each unique song as recommendation score
        train_data_grouped = train_data.groupby([self.item_id]).agg({self.user_id: 'count'}).reset_index()
        train_data_grouped.rename(columns = {'reviewerID': 'score'},inplace=True)
        print(train_data_grouped.columns)
        #Sort the songs based upon recommendation score
        train_data_sort = train_data_grouped.sort_values(['score', self.item_id], ascending = [0,1])
    
        #Generate a recommendation rank based upon score
        train_data_sort['Rank'] = train_data_sort['score'].rank(ascending=0, method='first')
        
        #Get the top 10 recommendations
        self.popularity_recommendations = train_data_sort.head(10)

    #Use the popularity based recommender system model to
    #make recommendations
    def recommend(self, user_id):    
        user_recommendations = self.popularity_recommendations
        
        #Add user_id column for which the recommendations are being generated
        user_recommendations['user_id'] = user_id
    
        #Bring user_id column to the front
        cols = user_recommendations.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        user_recommendations = user_recommendations[cols]
        
        return user_recommendations

In [ ]:
pm = popularity_recommender_py()
pm.create(Train, 'reviewerID', 'Product_ID')

In [ ]:
reviewr_id='A32HSNCNPRUMTR'
pm.recommend(reviewr_id)